# Chapter 5 Higher-Order Functions
Christoph Windheuser    
May, 2022   
Python examples of chapter 5 Higher-Order Functions (page 141 ff) in the book *Learning Spark*

In [1]:
# Import required python spark libraries
import findspark
import pyspark

from pyspark.conf import SparkConf
from pyspark.context import SparkContext

from pyspark.sql.types import *
from pyspark.sql.functions import *
# from pyspark.sql.functions import col, expr, when, concat, lit, avg, desc
from pyspark.sql import SparkSession
from pyspark.sql import Row


In [ ]:
# Only use when running a Jupyter Notebook, don't use it when starting with pyspark!
# Connect Jupyter Notebook with the Spark application and create Spark Context
findspark.init()
sc = pyspark.SparkContext(appName="chapter_4_HOF")


In [2]:
#create a SparkSession

spark = (SparkSession \
         .builder \
         .enableHiveSupport() \
         .config("spark.sql.catalogImplementation","hive") \
         .appName("Chapter_4_HOF") \
         .getOrCreate())


# Define a DataFrame

In [3]:
schema = StructType([StructField("celsius", ArrayType(IntegerType()))])


In [4]:
t_list = [[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]


In [5]:
t_c = spark.createDataFrame(t_list, schema)

In [6]:
t_c.createOrReplaceTempView("tC")

In [7]:
t_c.show()

+--------------------+
|             celsius|
+--------------------+
|[35, 36, 32, 30, ...|
|[31, 32, 34, 55, 56]|
+--------------------+



# Using the Higher-Order Function *transform()*
The `transform()` function produces an array by applying a function to each element of the input array.

In [8]:
# Calculate Fahrenheit from Celsius for an array of temperatures
spark.sql("""
SELECT celsius,
transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit
FROM tC
""").show()


+--------------------+--------------------+
|             celsius|          fahrenheit|
+--------------------+--------------------+
|[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
|[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+



# *filter()*
The function `filter()` produces an array consisting of only the elements of the input array for which the Boolean funciton is `true`.

In [9]:
# Filter all temperatures > 38C for an array of temperatures

spark.sql("""
SELECT celsius,
filter(celsius, t -> t > 38) as high
FROM tC
""").show()


+--------------------+--------+
|             celsius|    high|
+--------------------+--------+
|[35, 36, 32, 30, ...|[40, 42]|
|[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+



# *exists()*
The function `exists()` returns `true` if the boolean function holds for any element in the input array.

In [10]:
spark.sql("""
SELECT celsius,
       exists (celsius, t -> t = 38) as threshold
FROM tC
""").show()


+--------------------+---------+
|             celsius|threshold|
+--------------------+---------+
|[35, 36, 32, 30, ...|     true|
|[31, 32, 34, 55, 56]|    false|
+--------------------+---------+



# *aggregate()*
In the book, the function `reduce()` is used, but throws an error. The function `aggregate()` provides the correct result.

In [11]:
# Calculate the average temperature of a row and convert it to Fahrenheit
spark.sql("""
SELECT celsius,
    aggregate(
        celsius,
        0,
        (t, acc) -> t + acc,
        acc -> (acc div size(celsius) * 9 div 5) + 32
        ) as avgFahrenheit
FROM tC
""").show()


+--------------------+-------------+
|             celsius|avgFahrenheit|
+--------------------+-------------+
|[35, 36, 32, 30, ...|           96|
|[31, 32, 34, 55, 56]|          105|
+--------------------+-------------+



# Common DataFrames and Spark SQL Operations
Page 144 ff

In [12]:
from pyspark.sql.functions import expr

In [13]:
# Define the file paths
tripdelaysFilePath = """../DB_Spark/LearningSparkV2/databricks-datasets/\
learning-spark-v2/flights/departuredelays.csv"""

airportsnaFilePath = """../DB_Spark/LearningSparkV2/databricks-datasets/\
learning-spark-v2/flights/airport-codes-na.txt"""


In [14]:
# Obtain airport data set
airportsna = (spark.read
             .format("csv")
             .options(header="true", inferSchema="true", sep="\t")
             .load(airportsnaFilePath))


In [15]:
# Create a temporary view from the DataFrame
airportsna.createOrReplaceTempView("airports_na")


In [16]:
# Obtain departure delays data set
departureDelays = (spark.read
                  .format("csv")
                  .options(header="true")
                  .load(tripdelaysFilePath))


In [17]:
# Cast delay and distance to INT
departureDelays =(departureDelays
                 .withColumn("delay", expr("CAST(delay as INT) as delay"))
                 .withColumn("distance", expr("CAST(distance as INT) as distance")))


In [18]:
# Create a temporary view from the DataFrame
departureDelays.createOrReplaceTempView("departureDelays")

In [19]:
# Create temporary small table
foo = (departureDelays
      .filter(expr("""origin == 'SEA' and destination == 'SFO' and
      date like '01010%' and delay > 0""")))

foo.createOrReplaceTempView("foo")


In [20]:
# Show the table airports_na
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



In [21]:
# Show the table departureDelays
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



In [22]:
# Show the table foo
spark.sql("SELECT * FROM foo").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



## Unions
Page 147 ff

In [23]:
# Union the two tables "deparetureDelays" and "foo" and creating a new table "bar"
bar = departureDelays.union(foo)
bar.createOrReplaceTempView("bar")


In [24]:
# Filter for SEA and SFO in the specific time range - the same filter criteria as for foo.
# This will result in 2x the entries of "foo", as these entries are part of the tables
# "departureDelays" and "foo"
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO'
                    AND date LIKE '01010%' AND delay > 0""")).show()


+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



## Joins
Page 148

By default, a Spark SQL join is an inner join, but other options are possible (see documentation).

The following code performs an inner joins between the two DataFrames `airportsna` and `foo`.

In [25]:
foo.join(
    airportsna,
    airportsna.IATA == foo.origin
    ).select("City", "State", "date", "delay", "distance", "destination").show()


+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



## Windowing

In [26]:
spark.sql("DROP TABLE IF EXISTS departureDelaysWindow")

DataFrame[]

In [27]:
spark.sql("""
    CREATE TABLE departureDelaysWindow AS \
    SELECT origin, destination, SUM(delay) AS TotalDelays \
    FROM   departureDelays \
    WHERE  origin IN('SEA','SFO','JFK') \
    AND    destination IN('SEA','SFO','JFK','DEN','ORD','LAX','ATL') \
    GROUP  BY origin, destination;
""")

DataFrame[]

In [28]:
spark.sql("SELECT * FROM departureDelaysWindow").show()

+------+-----------+-----------+
|origin|destination|TotalDelays|
+------+-----------+-----------+
|   JFK|        ORD|       5608|
|   JFK|        SFO|      35619|
|   JFK|        DEN|       4315|
|   JFK|        ATL|      12141|
|   JFK|        SEA|       7856|
|   JFK|        LAX|      35755|
|   SEA|        LAX|       9359|
|   SFO|        ORD|      27412|
|   SFO|        DEN|      18688|
|   SFO|        SEA|      17080|
|   SEA|        SFO|      22293|
|   SFO|        ATL|       5091|
|   SEA|        DEN|      13645|
|   SEA|        ATL|       4535|
|   SEA|        ORD|      10041|
|   SFO|        JFK|      24100|
|   SFO|        LAX|      40798|
|   SEA|        JFK|       4667|
+------+-----------+-----------+



Using the window function `dense_ran()` to caluclate the tree top delayed destinations per origin airport:

In [29]:
spark.sql("""
SELECT origin, destination, TotalDelays, rank
FROM (
    SELECT origin, destination, TotalDelays, dense_rank()
    OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank
    FROM departureDelaysWindow
    ) t
    WHERE rank <= 3
""").show()


+------+-----------+-----------+----+
|origin|destination|TotalDelays|rank|
+------+-----------+-----------+----+
|   JFK|        LAX|      35755|   1|
|   JFK|        SFO|      35619|   2|
|   JFK|        ATL|      12141|   3|
|   SEA|        SFO|      22293|   1|
|   SEA|        DEN|      13645|   2|
|   SEA|        ORD|      10041|   3|
|   SFO|        LAX|      40798|   1|
|   SFO|        ORD|      27412|   2|
|   SFO|        JFK|      24100|   3|
+------+-----------+-----------+----+



## Modifications
Page 151 ff

### Adding new columns
Start with our small DataFrame example:

In [31]:
foo.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



Add a new column to the DataFrame:

In [32]:
foo2 = (foo.withColumn(
        "status",
        expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END")
))

In [33]:
foo2.show()

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|Delayed|
|01010955|  104|     590|   SEA|        SFO|Delayed|
|01010730|    5|     590|   SEA|        SFO|On-time|
+--------+-----+--------+------+-----------+-------+



### Dropping columns

In [35]:
foo3 = foo2.drop("delay")
foo3.show()

+--------+--------+------+-----------+-------+
|    date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
|01010710|     590|   SEA|        SFO|Delayed|
|01010955|     590|   SEA|        SFO|Delayed|
|01010730|     590|   SEA|        SFO|On-time|
+--------+--------+------+-----------+-------+



### Renaming columns

In [36]:
foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

+--------+--------+------+-----------+-------------+
|    date|distance|origin|destination|flight_status|
+--------+--------+------+-----------+-------------+
|01010710|     590|   SEA|        SFO|      Delayed|
|01010955|     590|   SEA|        SFO|      Delayed|
|01010730|     590|   SEA|        SFO|      On-time|
+--------+--------+------+-----------+-------------+



### Pivoting
Pivoting means swapping the columns for the rows in a table.

In [37]:
from_SEA_df = spark.sql("""
    SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
    FROM   departureDelays
    WHERE  origin = 'SEA'
""")


In [38]:
from_SEA_df.show()

+-----------+-----+-----+
|destination|month|delay|
+-----------+-----+-----+
|        ORD|    1|   92|
|        JFK|    1|   -7|
|        DFW|    1|   -5|
|        MIA|    1|   -3|
|        DFW|    1|   -3|
|        DFW|    1|    1|
|        ORD|    1|  -10|
|        DFW|    1|   -6|
|        DFW|    1|   -2|
|        ORD|    1|   -3|
|        ORD|    1|    0|
|        DFW|    1|   23|
|        DFW|    1|   36|
|        ORD|    1|  298|
|        JFK|    1|    4|
|        DFW|    1|    0|
|        MIA|    1|    2|
|        DFW|    1|    0|
|        DFW|    1|    0|
|        ORD|    1|   83|
+-----------+-----+-----+
only showing top 20 rows



In [39]:
from_SEA_df2 = spark.sql("""
    SELECT * FROM (
    SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
    FROM   departureDelays WHERE  origin = 'SEA'
    )
    PIVOT (
        CAST(AVG(delay) AS DECIMAL(4, 2)) AS AvgDelay, MAX(delay) AS MaxDelay
        FOR month IN (1 JAN, 2 FEB)
    )
    ORDER BY destination
""")

In [40]:
from_SEA_df2.show()

+-----------+------------+------------+------------+------------+
|destination|JAN_AvgDelay|JAN_MaxDelay|FEB_AvgDelay|FEB_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       19.86|         316|       11.42|          69|
|        ANC|        4.44|         149|        7.90|         141|
|        ATL|       11.98|         397|        7.73|         145|
|        AUS|        3.48|          50|       -0.21|          18|
|        BOS|        7.84|         110|       14.58|         152|
|        BUR|       -2.03|          56|       -1.89|          78|
|        CLE|       16.00|          27|        null|        null|
|        CLT|        2.53|          41|       12.96|         228|
|        COS|        5.32|          82|       12.18|         203|
|        CVG|       -0.50|           4|        null|        null|
|        DCA|       -1.15|          50|        0.07|          34|
|        DEN|       13.13|         425|       12.95|         625|
|        D